In [1]:
import z from "zod";
import { zodToJsonSchema } from "zod-to-json-schema";
import { ChatOpenAI } from "langchain/chat_models/openai";

import { load } from "dotenv";
const env = await load();
const process = { env };
function getCurrentWeather({ location, unit = "fahrenheit" }) {
  console.log("123");
  const weather_info = {
    location: location,
    temperature: "72",
    unit: unit,
    forecast: ["sunny", "windy"],
  };
  return JSON.stringify(weather_info);
}
const getCurrentWeatherSchema = z.object({
  location: z.string().describe("城市或地区。例如：'北京'"),
  unit: z.enum(["celsius", "fahrenheit"]).optional().describe("温度单位。默认值：'fahrenheit'"),
});
const model = new ChatOpenAI({
  configuration: {
    baseURL: process.env.baseURL,
  },
  model: process.env.model,
});
const modelWithTools = model.bind({
  tools: [
    {
      type: "function",
      function: {
        name: "getCurrentWeather",
        description: "Get the current weather in a given location",
        parameters: zodToJsonSchema(getCurrentWeatherSchema),
      },
    },
  ],
  // tool_choice: {
  //   type: "function",
  //   function: {
  //     name: "getCurrentWeather",
  //   }
  // }
});
await modelWithTools.invoke("上海的天气怎么样");


[WARNING]: Importing from "langchain/chat_models/openai" is deprecated.

Instead, please add the "@langchain/openai" package to your project with e.g.

    $ npm install @langchain/openai

and import from "@langchain/openai".

This will be mandatory after the next "langchain" minor version bump to 0.2.


In [ ]:
import { ChatPromptTemplate } from "@langchain/core/prompts";
const prompt = ChatPromptTemplate.fromMessages([
  ["system", "你是一个乐于助人的助手"],
  ["human", "{input}"],
]);
const chain = prompt.pipe(modelWithTools);
let res = await chain.invoke({
  input: "上海的天气怎么样",
});
console.log(res);
